In [1]:
# KEYWORDS: MinHash(LSH) 

In [135]:
import numpy as np
import pandas as pd 
import string
import random
import csv
import time
import math

## Reading Data Files 

In [3]:
# The target file includes the original sequences that encodes the data. 
with open('targetBlawat et al.LDPC.txt') as f:
    reads = f.readlines()
for i in range(0,len(reads)):
    reads[i] = reads[i][0:len(reads[i])-1]
reads 

['...',
 ...]

In [4]:
np.size(reads)

9608

In [5]:
# The errors_shuffled file include the erroneous copies shuffled. 
with open('errors_shuffled.txt') as f:
    reads_err = f.readlines()
for i in range(0,len(reads_err)):
    reads_err[i] = reads_err[i][0:len(reads_err[i])-1]
# reads_err = reads_err[0:1000] 

In [6]:
np.size(reads_err)

1196656

In [7]:
# evyat.txt include the erroneous copies clustered and labeled. 
with open('evyat.txt') as f:
    reads_cl = f.readlines()
for i in range(0,len(reads_cl)):
    reads_cl[i] = reads_cl[i][0:len(reads_cl[i])-1]
reads_cl

['TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 '*****************************',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGCGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGAAACCCACACTGGGTTGTACTTGGTCAAA',
 'TGAATAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTTGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAACACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAAAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA',
 'TGAATAAAGACGAGTCCCGGAAACCCACACT

## Functions


In [166]:
def hash_fun_ind(x,a,w,l): 
    cnt = 0 
    for i in range(0,len(x)):
        if x[i] == a[cnt]:
            cnt += 1  
        else:
            cnt = 0 
        if cnt == w:
            return i-w+1 
    return -1

def hash_fun(x,a,w,l):
    ind = hash_fun_ind(x,a,w,l)
    return x[ind:min(len(x),ind+w+l)]

def ind_st(st):
    #All 3-grams: AAA,AAG,AAC,AAT,AGA,AGC,...,TTA,TTG,TTC,TTT
    # A-0, G-1, C-2, T-3
    # index of CAT = Decimal representaion of (203)
    N_q = {"A":0,"C":1,"G":2,"T":3}
    dec = 0
    for i in range(0,len(st)):
        dec += N_q[st[i]]*(4**(len(st)-i-1))
    return dec 
    
def bin_sig(x,q):
    bs = [0]*(4**q) 
    for i in range(0,len(x)-q+1):
        st = x[i:i+q]
        bs[ind_st(st)] = 1
    return bs 
        
def ham_dis(x,y):
    dis = 0
    for i in range(0,len(x)):
        if(x[i]!= y[i]):
            dis += 1
    return dis 

def rand_perm(w):
    return ''.join(random.choice('ACGT') for _ in range(w))

def merge(x,y,parent):
    parent[x]= min(parent[x],parent[y])
    parent[y]= min(parent[x],parent[y])
    
def rep_find(inp):
    temp = inp
    cnt = 0
    while(parent[temp]!=temp and cnt<10):
        cnt += 1
        temp=parent[temp]
    return temp 

def edit_dis(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

def find_in_arr(el,parent):
    cnt = 0
    for i in range(0,len(parent)):
        if parent[i] == el: 
            cnt += 1 
    return cnt 
        
        
    

In [107]:
rand_perm(4)

'TCTG'

In [108]:
hash_fun("abc","c",1,2)

'c'

In [109]:
hash_fun(reads_err[0],"GA",2,2)

'GAGA'

In [110]:
bin_sig("TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA",2)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]

In [111]:
ham_dis("ACT","ATG")

2

In [179]:
q =  3
x = "CTATCTTCACAAATACCTGTTCCTGCTCGCCTAGTTCTGCCTGGACGACT"
y = "TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA"
ham_dis(bin_sig(x,q),bin_sig(y,q))

28

In [180]:
edit_dis(x,y)

26

In [150]:
x = "TGAATAAAAAGACGAGTCCCGGAAACCCACACTGGTTGTACTTGGTCAAA"
y = "TGAATAAAAAGACTCCCGGAAACACACTGGTTGTACTTGGTCAAA"
ham_dis(bin_sig(x,q),bin_sig(y,q))

7

In [151]:
edit_dis(x,y)

5

## A Local Computaion Step of the Algorithm 

In [126]:
# GLOBAL CLUTERING DENOTED BY C_til
# Parent stores cluster id corresponding to each read
# Disjoint set Union Problem 

C_til  = [0]*(len(reads_err))
parent = [0]*(len(reads_err))
for i in range(0,len(reads_err)):
    C_til[i] = (i,reads_err[i]) 
    parent[i] = i
C_til

[(0, 'GGGAGACTCGCAATATAGAATACATACTTGACGTAGCAGTCTTTTATGGA'),
 (1, 'GCAGAGGATTCCTGGGCCGCTGATCCAGTTAACACATGTGATGGCCACAC'),
 (2, 'ATCATGGTCAAGAGGTTGATTTCGATCAACTGAGCATCCCGTCTGCGGTTG'),
 (3, 'TCATCTCTAAGCGGCGAGCTACAATCCGCTCCGAAGCGCTCCCTAGTAAC'),
 (4, 'ATCGGTACCCTCAAACTAGCATGGAACTTCTATCGGATCGCTGGCAGATA'),
 (5, 'GAGTGGTCTCGAAGACATGCAACCCAGCACATACAACACCCGAGCGGTAAT'),
 (6, 'TCGCTGTTAAATCGTCACACCACGTCAACTATACGTAGTACCTCCCTGAC'),
 (7, 'AGGCCGCTTATAGAGGACGTCTGTTCAGGTCATCGGTATGTCTCGTGAG'),
 (8, 'GTATCCTACTCAGCTTAAGGGGCACACGCCAGCGCATGCACACACTGCTCC'),
 (9, 'CTATCTTCACAAATACCTGTTCCTGCTCGCCTAGTTCTGCCTGGACGACT'),
 (10, 'CTACCCAGTAGCGCATACGTTAGCTATTATGAGGCTTGATGGTCGAAGTGG'),
 (11, 'TAGGGTTGCATCTTTGCATAATAAACTTGGCCATAGCGTGTCTCCCTCCC'),
 (12, 'GTTGTCAAGGGACATGGATTTGCCCCTCGGGATGTATTCAGGGATTCA'),
 (13, 'CGCATTGGGTAGGACCAACATCTGCTCCGCATGTGAGGGTTGATATGATG'),
 (14, 'GGTTATGGCTACTTGTCTACGGTGACTAGTTGACCGCTTAGCCTCCCGGT'),
 (15, 'CCATGTGAGGAGACTGGCTACCTCGGCAGGTAAGTTCGGGCATAACCGATT'),
 (16, 'ACTTGAAACGTAGACGCACGATATC

In [137]:
# Step 1: Sample a Hash Function 
# Step 2: Compute Hash of a random representative of each cluster 
# Step 3: Merge two clusters with same hash if ham_dis(bin_sig(x),bin_sig(y)) <= th_high
a = "GGA"
w = len(a)
l = math.ceil(math.log(1196656,4))
hash_C_til = [0]*(len(reads_err))

for i in range(0,len(C_til)):
    hash_C_til[i] = (i,hash_fun(C_til[i][1],a,w,l))

In [136]:
l = math.ceil(math.log(1196656,4))
l

11

In [138]:
hash_C_til

[(0, 'GGA'),
 (1, 'GGATTCCTGGGCCG'),
 (2, ''),
 (3, ''),
 (4, 'GGAACTTCTATCGG'),
 (5, ''),
 (6, ''),
 (7, 'GGACGTCTGTTCAG'),
 (8, ''),
 (9, 'GGACGACT'),
 (10, ''),
 (11, ''),
 (12, 'GGATTTGCCCCTCG'),
 (13, 'GGACCAACATCTGC'),
 (14, ''),
 (15, 'GGAGACTGGCTACC'),
 (16, ''),
 (17, 'GGATACGCAAAATA'),
 (18, 'GGACTCATATGTGC'),
 (19, 'GGAAGACCTGCGCA'),
 (20, ''),
 (21, ''),
 (22, ''),
 (23, 'GGACGTTTTAAAAA'),
 (24, ''),
 (25, 'GGACGCAGCGCATG'),
 (26, ''),
 (27, 'GGAGCGCCAATCGT'),
 (28, ''),
 (29, 'GGAGTACTGTTGGA'),
 (30, 'GGAACTTGAGCGTT'),
 (31, ''),
 (32, 'GGAAGTGATCGCCG'),
 (33, 'GGAGCCTCAAGATA'),
 (34, 'GGACCTTACGTCCA'),
 (35, 'GGATTGCACTCATA'),
 (36, ''),
 (37, ''),
 (38, ''),
 (39, 'GGAAACATTCAAT'),
 (40, ''),
 (41, 'GGAATAAA'),
 (42, ''),
 (43, ''),
 (44, 'GGACGATTCCCCAG'),
 (45, ''),
 (46, 'GGAGATG'),
 (47, 'GGATGACTTGCGCT'),
 (48, 'GGAATTCGATTCCT'),
 (49, 'GGATGGGAACACAA'),
 (50, ''),
 (51, 'GGATTCCTGGGCCG'),
 (52, ''),
 (53, ''),
 (54, ''),
 (55, ''),
 (56, ''),
 (57, ''),
 (58, ''),


In [139]:
len("GGACGCCAATCGGG")

14

In [140]:
hash_C_til.sort(key=lambda x:x[1])
hash_C_til

[(2, ''),
 (3, ''),
 (5, ''),
 (6, ''),
 (8, ''),
 (10, ''),
 (11, ''),
 (14, ''),
 (16, ''),
 (20, ''),
 (21, ''),
 (22, ''),
 (24, ''),
 (26, ''),
 (28, ''),
 (31, ''),
 (36, ''),
 (37, ''),
 (38, ''),
 (40, ''),
 (42, ''),
 (43, ''),
 (45, ''),
 (50, ''),
 (52, ''),
 (53, ''),
 (54, ''),
 (55, ''),
 (56, ''),
 (57, ''),
 (58, ''),
 (60, ''),
 (62, ''),
 (63, ''),
 (64, ''),
 (69, ''),
 (70, ''),
 (72, ''),
 (75, ''),
 (76, ''),
 (77, ''),
 (79, ''),
 (80, ''),
 (81, ''),
 (84, ''),
 (85, ''),
 (86, ''),
 (87, ''),
 (88, ''),
 (89, ''),
 (90, ''),
 (91, ''),
 (92, ''),
 (93, ''),
 (95, ''),
 (98, ''),
 (100, ''),
 (102, ''),
 (104, ''),
 (106, ''),
 (107, ''),
 (110, ''),
 (111, ''),
 (112, ''),
 (113, ''),
 (114, ''),
 (118, ''),
 (119, ''),
 (120, ''),
 (121, ''),
 (122, ''),
 (124, ''),
 (129, ''),
 (130, ''),
 (132, ''),
 (133, ''),
 (136, ''),
 (137, ''),
 (138, ''),
 (143, ''),
 (144, ''),
 (145, ''),
 (146, ''),
 (149, ''),
 (150, ''),
 (151, ''),
 (152, ''),
 (154, ''),
 (156

In [152]:
q = 3
th_high = 10
th_low  = 5
cnt = 0
r = 5

start = time.time()
for i in range(0,len(reads_err)-1):
    if(hash_C_til[i][1] == ""):
        continue 
    else:
        if hash_C_til[i][1] == hash_C_til[i+1][1]:
            if rep_find(i) == rep_find(i+1):
                continue 
            x = reads_err[hash_C_til[i][0]]
            y = reads_err[hash_C_til[i+1][0]]
            if((ham_dis(bin_sig(x,q),bin_sig(y,q)) <= th_low) or ((ham_dis(bin_sig(x,q),bin_sig(y,q)) <= th_low) and edit_dis(x,y)<=r)):
                cnt +=  1
                merge(hash_C_til[i][0],hash_C_til[i+1][0],parent)
print(cnt)

end = time.time()
print(end-start)

351249
95.66822719573975


In [156]:
parent

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 1,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 48,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 59,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 140,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 1

In [157]:
rep_find(67)

48

# Clustering Algorithm

In [163]:
with open('errors_shuffled.txt') as f:
    reads_err = f.readlines()
for i in range(0,len(reads_err)):
    reads_err[i] = reads_err[i][0:len(reads_err[i])-1]
    
C_til  = [0]*(len(reads_err))
parent = [0]*(len(reads_err))
for i in range(0,len(reads_err)):
    C_til[i] = (i,reads_err[i]) 
    parent[i] = i

local_comm = 100
q = 3
th_high = 15
th_low  = 8
r = 8
w = 3
l = math.ceil(math.log(np.size(reads_err),4))

cnt = 0


for lcl_step in range(0,local_comm):
    a = rand_perm(w)
    hash_C_til = [0]*(len(reads_err))

    for i in range(0,len(C_til)):
        hash_C_til[i] = (i,hash_fun(C_til[i][1],a,w,l))
    
    hash_C_til.sort(key=lambda x:x[1])

    cnt =0
    for i in range(0,len(reads_err)-1):
        if(hash_C_til[i][1] == ""):
            continue 
        else:
            if hash_C_til[i][1] == hash_C_til[i+1][1]:
                if rep_find(i) == rep_find(i+1):
                    continue 
                x = reads_err[hash_C_til[i][0]]
                y = reads_err[hash_C_til[i+1][0]]
                if((ham_dis(bin_sig(x,q),bin_sig(y,q)) <= th_low) or ((ham_dis(bin_sig(x,q),bin_sig(y,q)) <= th_low) and edit_dis(x,y)<=r)):
                    cnt +=  1
                    merge(hash_C_til[i][0],hash_C_til[i+1][0],parent)
                    
    print(lcl_step,cnt)

0 28953
1 28439
2 28536
3 27626
4 26553
5 28952
6 27250
7 27602
8 29570
9 28119
10 28890
11 29879
12 27549
13 28795
14 28867
15 30345
16 27996
17 29542
18 24814
19 29013
20 28248
21 28374
22 29959
23 27200
24 29624
25 31057
26 28054
27 28186
28 31010
29 30914
30 30349
31 28315
32 27358
33 29058
34 30291
35 28764
36 29745
37 29918
38 28374
39 30568
40 29971
41 29745
42 25904
43 28629
44 29079
45 27455
46 28411
47 27952
48 27408
49 27830
50 30649
51 27785
52 28862
53 30081
54 26562
55 29004
56 29695
57 29397
58 27802
59 28123
60 29164
61 30649
62 28877
63 28552
64 29037
65 28248
66 25903
67 31830
68 27709
69 28539
70 29943
71 28281
72 30218
73 26065
74 28473
75 28794
76 26565
77 29959
78 29013
79 29119
80 28918
81 29974
82 29066
83 28406
84 28091
85 29851
86 30327
87 30502
88 28446
89 30200
90 30914
91 29119
92 28747
93 28943
94 27131
95 27952
96 27573
97 30577
98 26093
99 27883


In [164]:
parent

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 1,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 48,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 59,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 140,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 136,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 1

In [165]:
np.savetxt("parent_file.txt", np.array(parent), fmt="%s")

In [176]:
find_in_arr(48,parent)

11

In [209]:
def display_parent(parent):
    clstr = {}
    for i in range(0,len(parent)):
        clstr[i]=[]
    for i in range(0,len(parent)):
        clstr[rep_find(i)].append(i)
    return clstr

In [210]:
clustering = display_parent(parent)

{0: [0, 2648, 6755],
 1: [1, 51],
 2: [2],
 3: [3, 3306, 6974, 11375, 11420],
 4: [4, 4359],
 5: [5,
  28408,
  35447,
  51195,
  58145,
  60221,
  63559,
  78533,
  80583,
  86636,
  86695,
  96196,
  112881,
  118727],
 6: [6,
  16334,
  17875,
  24942,
  28048,
  30206,
  38781,
  42971,
  43184,
  47077,
  50715,
  57777,
  60781,
  77933,
  78330,
  83690,
  88120,
  89931,
  92828],
 7: [7, 17547, 30356, 34644, 46426],
 8: [8],
 9: [9, 12902, 17813, 21876],
 10: [10,
  2188,
  6901,
  11557,
  13606,
  19972,
  21216,
  26083,
  35087,
  36602,
  39561,
  43861,
  60656,
  62786,
  71821,
  74750,
  78253],
 11: [11, 4171, 6482],
 12: [12, 7651, 13554],
 13: [13,
  2069,
  2988,
  7732,
  14658,
  17181,
  19797,
  31711,
  47311,
  49844,
  52780,
  53130,
  63482,
  67614,
  80513,
  106978,
  111937,
  113034,
  114788,
  137669,
  137810,
  151991,
  158780,
  159315,
  160409,
  162931,
  164196,
  165142,
  167266,
  167775,
  168117,
  170082,
  172880,
  181746,
  182955,

In [211]:
clstr[1095]

NameError: name 'clstr' is not defined